In [17]:
import tweepy
from kafka import KafkaProducer
import logging

In [18]:
consumerKey = 'pQg0kPJU9g0h3rG8TsUV97rrY'
consumerSecret = 'pcjYZNKn7a1jcjGIuZ03REP6mXkQefWtlW3SPOhp13jv19u0yl'
accessToken = '1377628522718638088-VYoYsCJgIaqD5bcxQSGqpd9yhNkj5U'
accessTokenSecret = 'bT9zOqLEPJSNiCC2Hp38tioMVOqyySDkhzly7a8BKmMCY'

In [19]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')
search_term = 'Chirine'
topic_name = 'skype'

In [20]:
def twitterAuth():
    # create the authentication object
    authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
    # set the access token and the access token secret
    authenticate.set_access_token(accessToken, accessTokenSecret)
    # create the API object
    api = tweepy.API(authenticate, wait_on_rate_limit=True)
    return api

In [21]:
class TweetListener(tweepy.Stream):

    def on_data(self, raw_data):
        logging.info(raw_data)
        producer.send(topic_name, value=raw_data)
        return True

    def on_error(self, status_code):
        if status_code == 420:
            # returning False in on_data disconnects the stream
            return False

    def start_streaming_tweets(self, search_term):
        self.filter(track=search_term, stall_warnings=True, languages=["en"])

In [23]:
if __name__ == '__main__':
    twitter_stream = TweetListener(consumerKey, consumerSecret, accessToken, accessTokenSecret)
    twitter_stream.start_streaming_tweets(search_term)
    # create the streaming object
    #api = twitterAuth() ['#python']

TypeError: __init__() takes 3 positional arguments but 4 were given